IMPORTANDO BIBLIOTECAS

In [23]:
import os
import random
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch.optim import Adam, SGD
import torch.nn as nn
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm


CONFIGURAÇÕES GERAIS

In [24]:
# Configurações gerais
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

IMPORTANDO DATA SET

In [25]:
# Caminhos do dataset
dataset_root = './DataSet'
training_dir = os.path.join(dataset_root, 'Training')
validation_dir = os.path.join(dataset_root, 'Validation')
testing_dir = os.path.join(dataset_root, 'Testing')

TRANSFORMAÇÃO DAS IMAGENS, CARREGAMENTO DO DATASET E CRIAÇÃO DOS DATA LOADERS

In [26]:
# Transformações para as imagens
transform = {
    'train': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=20),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val_test': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

# Carregar datasets
datasets_dict = {
    'train': datasets.ImageFolder(root=training_dir, transform=transform['train']),
    'val': datasets.ImageFolder(root=validation_dir, transform=transform['val_test']),
    'test': datasets.ImageFolder(root=testing_dir, transform=transform['val_test'])
}

# Criar DataLoaders
def create_dataloaders(batch_size):
    return {
        'train': DataLoader(datasets_dict['train'], batch_size=batch_size, shuffle=True),
        'val': DataLoader(datasets_dict['val'], batch_size=batch_size, shuffle=False),
        'test': DataLoader(datasets_dict['test'], batch_size=batch_size, shuffle=False)
    }
    

CONFIGURAÇÃO DOS MODELOS

In [ ]:
# Configurar modelos
num_classes = len(datasets_dict['train'].classes)

resnet_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, num_classes)

# DenseNet121
densenet_model = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
densenet_model.classifier = nn.Linear(densenet_model.classifier.in_features, num_classes)

# Inception v3
inception_model = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT, aux_logits=True)
inception_model.fc = nn.Linear(inception_model.fc.in_features, num_classes)

FUNÇÃO DE TREINAMENTO

In [28]:
# Função de treinamento
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10, model_name="model"):
    model = model.to(device)

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in tqdm(train_loader, desc=f"Treinando - Epoch {epoch+1}/{epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)

            if isinstance(outputs, tuple):  # Para Inception v3
                main_output, aux_output = outputs
                loss = criterion(main_output, labels) + 0.4 * criterion(aux_output, labels)
            else:
                loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs[0] if isinstance(outputs, tuple) else outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_accuracy = 100 * correct / total
        train_losses.append(running_loss / len(train_loader))
        train_accuracies.append(train_accuracy)

        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc="Validando"):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                if isinstance(outputs, tuple):
                    main_output = outputs[0]
                    loss = criterion(main_output, labels)
                else:
                    loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = torch.max(main_output if isinstance(outputs, tuple) else outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        val_accuracy = 100 * correct / total
        val_losses.append(val_loss / len(val_loader))
        val_accuracies.append(val_accuracy)

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%")
        print(f"Validation Loss: {val_loss/len(val_loader):.4f}, Validation Accuracy: {val_accuracy:.2f}%\n")

    torch.save({
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_accuracies': train_accuracies,
        'val_accuracies': val_accuracies
    }, f"{model_name}_metrics.pth")

    return model

FUNÇÃO DE AVALIAÇÃO E PLOTAGEM DA MATRIZ DE CONFUSÃO

In [ ]:
# Função de avaliação e matriz de confusão
def evaluate_model(model, test_loader, model_name="model"):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            if isinstance(outputs, tuple):
                outputs = outputs[0]  # Usar apenas a saída principal
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds, average='weighted')
    rec = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1 Score: {f1:.4f}")

    cm = confusion_matrix(all_labels, all_preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=datasets_dict['train'].classes)
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f"Matriz de Confusão - {model_name}")
    plt.savefig(f"{model_name}_confusion_matrix.png")
    plt.show()

TREINO E AVALIAÇÃO DO MODELO RESNET

In [ ]:
# Treinar e avaliar ResNet
criterion = nn.CrossEntropyLoss()
optimizer = Adam(resnet_model.parameters(), lr=0.001)
trained_resnet = train_model(resnet_model, dataloaders['train'], dataloaders['val'], criterion, optimizer, epochs=5, model_name="ResNet18")
evaluate_model(trained_resnet, dataloaders['test'], model_name="ResNet18")


TREINO E AVALIAÇÃO DO MODELO DENSENET

In [ ]:
# Treinar e avaliar DenseNet
optimizer = Adam(densenet_model.parameters(), lr=0.001)
trained_densenet = train_model(densenet_model, dataloaders['train'], dataloaders['val'], criterion, optimizer, epochs=5, model_name="DenseNet121")
evaluate_model(trained_densenet, dataloaders['test'], model_name="DenseNet121")


TREINO E AVALIAÇÃO DO MODELO INCEPTION V3

In [ ]:

# Treinar e avaliar Inception
optimizer = Adam(inception_model.parameters(), lr=0.001)
trained_inception = train_model(inception_model, dataloaders['train'], dataloaders['val'], criterion, optimizer, epochs=5, model_name="InceptionV3")
evaluate_model(trained_inception, dataloaders['test'], model_name="InceptionV3")
